In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
key_path = "./creds/service_acc_auth.json"
credentials = service_account.Credentials.from_service_account_file(key_path, scopes = ["https://www.googleapis.com/auth/cloud-platform"])
bigquery_client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [2]:
# query new data
union_weeks = """
SELECT * , 
FROM `scanning-database.scanning_data_backup.W23` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W24` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W25` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W26` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W27` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W28` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W29`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W30`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W31`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W32`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W33`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W34`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W35`
"""

In [3]:
# Overwriting old PowerBI source data
job_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10,
destination = 'scanning-database.PowerBI.PowerBI',
write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
) # set query limit to 10G
query_job = bigquery_client.query(union_weeks, job_config)  # API request
rows = query_job.result()  # Waits for query to finish
data_in_use = rows.to_dataframe()

In [4]:
data_in_use.head()

,Start_Time,Finish_Time,Activity,Seq_Code,Recipe_Name,Break_Reasons,Missing_Ingredients,Kitting_Line,Assembly_Batch,Event_Shift,Team_Leader,Pickers_Count,Time_Consumption,Week
0,2020-08-22 21:04:11+00:00,2020-08-22 21:04:39+00:00,Production,None,Q4,None,None,KL19,None,Saturday Afternoon,Dummy,0,0.47,2020-35
1,2020-05-31 11:40:00+00:00,2020-05-31 11:41:00+00:00,TBD,None,R4,None,None,KL18,None,Sunday Morning,Dummy,0,0.45,2020-23
2,2020-08-01 07:39:17+00:00,2020-08-01 09:52:59+00:00,Factory Closed,None,C4,None,None,KL3,None,Saturday Morning,Leon,0,133.70,2020-32
3,2020-06-06 08:17:00+00:00,2020-06-06 08:17:00+00:00,Preparation/Changeover,None,J2,None,None,KL6,None,Saturday Morning,Test,0,0.22,2020-24
4,2020-06-06 11:22:00+00:00,2020-06-06 15:02:00+00:00,Break,None,A2,shift break,None,KL15,None,Saturday Afternoon,Kim,0,219.65,2020-24


In [5]:
data_in_use.shape

(91775, 14)